In [15]:
import toml
from typing import List
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import WebBaseLoader
from pydantic import BaseModel, Field

from rich import print as rprint

In [ ]:
llm_config = toml.load("../config.toml")["configs"][0]
llm = AzureChatOpenAI(
    api_version=llm_config["api_version"],
    azure_deployment=llm_config["deployment_name"],
    api_key=llm_config["api_key"],
    azure_endpoint=llm_config["api_endpoint"],
    model=llm_config["model_name"],
    temperature=0,
)
llm.invoke("hi")

In [8]:
def web_scrape_for_content(source_url: str) -> str:
    loader = WebBaseLoader(source_url)
    docs = loader.load()
    return "\n\n".join(
        [
            f'<Document name="{doc.metadata.get("title", "")}">\n{doc.page_content}\n</Document>'
            for doc in docs
        ]
    )

In [16]:
class FNSActiveDisasterResponseInfo(BaseModel):
    states_recieving_assistance: List[str] = Field(
        description="A list of states that are recieving assistance"
    )

In [12]:
url_to_scrape = "https://www.fns.usda.gov/disaster"
web_content = web_scrape_for_content(url_to_scrape)

In [ ]:
prompt_template = """
You are an expert at scraping websites for information. You will only respond with information from the given website.

Text: {text}
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["text"],
)

chain = prompt | llm.with_structured_output(FNSActiveDisasterResponseInfo, method="function_calling)

response = chain.invoke({"text": web_content}")

rprint(response)